***
### Read in structure with `create_builder_from_file`

In [ ]:

# ! Way too complex with builder, do it via ase reading in QE input
def add_structure_data(df_in):
    df_out = df_in.copy(deep=True)
    # iterate over tuples of dataframe rows
    structure_datas = []
    for index, row in df_out.iterrows():
        # df_out.at[index, 'structureData'] = 
        try:
            created_builder = create_builder_from_file(
                input_folder=row['abs_path'],
                input_file_name=row['filename'],
                code='qe-6.4.1-pw',
                metadata={},
                pseudo_folder_path=pseudo_dir
                )
            structure_data = created_builder['structure']
        except FileNotFoundError:
            # ! Missing PP (for Ni, PBEsol USPP)
            structure_data = None
        except ParsingError:
            # ! No atomic coordinates in input file, e.g. for DOS
            structure_data = None
        except ValueError:
            print(repr(row['abs_path']))
            print(row['filename'])
            structure_data = None
            raise
        
        structure_datas.append(structure_data)
    df_out['structuredata'] = structure_datas 
    return df_out
    
devel_df = spinel_df_iurii.loc[spinel_df_iurii['abs_path'].str.contains('LixMn2O4/DFT\+U/LiMn2O4/1_vcrelax')]
devel_df = spinel_df_iurii.copy(deep=True)
devel_df.shape
print(devel_df['abs_path'])

devel_df = add_structure_data(devel_df)

devel_df.head()